In [1]:
!python -m pip install --upgrade pip
!python -m pip install autogluon

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinu

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
train_data = TabularDataset('TrainSet_klassification.csv')
train_data.head()

,lda_1,income
0,3.808526,1
1,-1.856677,0
2,1.368470,0
3,-1.854464,0
4,-0.873393,0


In [4]:
label = 'income'
train_data[label].describe()

,income
count,23167.000000
mean,0.434541
std,0.495707
min,0.000000
25%,0.000000
50%,0.000000
75%,1.000000
max,1.000000


In [5]:
test_data = TabularDataset('TestSet_klassification.csv')
test_data.head()

,lda_1,income
0,-1.319791,0
1,-2.273363,0
2,0.556461,1
3,-1.585872,0
4,-1.672495,0


In [6]:
#runtime 4h
predictor = TabularPredictor(label=label,path="agModels_perdict_income").fit(train_data, time_limit=14400, presets='best_quality')

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       11.20 GB / 12.67 GB (88.3%)
Disk Space Avail:   190.14 GB / 225.83 GB (84.2%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stac

In [7]:
predictor.evaluate(test_data)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


{'accuracy': 0.8070046082949309,
 'balanced_accuracy': 0.7607211026395435,
 'mcc': 0.47314593276454425,
 'roc_auc': 0.8588728948506228,
 'f1': 0.5882815572158867,
 'precision': 0.5165745856353591,
 'recall': 0.6831050228310502}

In [8]:
predictor.leaderboard(test_data)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,LightGBMXT_BAG_L1,0.812719,0.874390,accuracy,0.038029,0.046547,30.202516,0.038029,0.046547,30.202516,1,True,3
1,WeightedEnsemble_L2,0.812719,0.874390,accuracy,0.040579,0.051148,30.685887,0.002549,0.004601,0.483371,2,True,5
2,LightGBM_BAG_L1,0.807373,0.873959,accuracy,0.054611,0.261275,38.818080,0.054611,0.261275,38.818080,1,True,4
3,LightGBMXT_BAG_L2,0.807005,0.874606,accuracy,0.461619,0.776376,104.826308,0.336409,0.332066,35.737196,2,True,6
4,WeightedEnsemble_L3,0.807005,0.874606,accuracy,0.464483,0.780817,105.319361,0.002864,0.004441,0.493053,3,True,7
5,KNeighborsUnif_BAG_L1,0.801106,0.854362,accuracy,0.016494,0.082666,0.034261,0.016494,0.082666,0.034261,1,True,1
6,KNeighborsDist_BAG_L1,0.790599,0.836492,accuracy,0.016077,0.053821,0.034255,0.016077,0.053821,0.034255,1,True,2


In [9]:
#leaderbaord download
import pandas as pd

leaderboard_df = predictor.leaderboard(test_data, extra_info=True)

print(leaderboard_df)

                   model  score_test  score_val eval_metric  pred_time_test  \
0      LightGBMXT_BAG_L1    0.812719   0.874390    accuracy        0.066384   
1    WeightedEnsemble_L2    0.812719   0.874390    accuracy        0.070321   
2        LightGBM_BAG_L1    0.807373   0.873959    accuracy        0.099798   
3      LightGBMXT_BAG_L2    0.807005   0.874606    accuracy        0.684295   
4    WeightedEnsemble_L3    0.807005   0.874606    accuracy        0.688039   
5  KNeighborsUnif_BAG_L1    0.801106   0.854362    accuracy        0.030122   
6  KNeighborsDist_BAG_L1    0.790599   0.836492    accuracy        0.032768   

   pred_time_val    fit_time  pred_time_test_marginal  pred_time_val_marginal  \
0       0.046547   30.202516                 0.066384                0.046547   
1       0.051148   30.685887                 0.003937                0.004601   
2       0.261275   38.818080                 0.099798                0.261275   
3       0.776376  104.826308               

In [10]:
from google.colab import files

leaderboard_df.to_csv('autogluon_leaderboard_classification_preprocessing.csv', index=False)

files.download('autogluon_leaderboard_classification_preprocessing.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:
#predictor scores download
import pandas as pd
from google.colab import files

#predict score
evaluation_results = predictor.evaluate(test_data)

#to df
evaluation_df = pd.DataFrame([evaluation_results])

print(evaluation_df)

#to CSV
evaluation_df.to_csv('autogluon_classification_predictorScore_preprocessing.csv', index=False)


files.download('autogluon_classification_predictorScore_preprocessing.csv')

   accuracy  balanced_accuracy       mcc   roc_auc        f1  precision  \
0  0.807005           0.760721  0.473146  0.858873  0.588282   0.516575   

     recall  
0  0.683105  


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>